In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import string
from sklearn.cluster import AgglomerativeClustering

Импортируем необходимый функционал. Для кластеризации будем использовать **AgglomerativeClustering** из-за его преимущества в скорости и в хорошей поддержке большого количества кластеров.

In [2]:
data_path = "."

In [3]:
data = pd.read_csv(f'{data_path}/JOB_LIST.csv', encoding='utf-8', encoding_errors='ignore')
data.dropna(inplace=True)
data = data['job_title'].to_numpy()

Выделяем учебную выборку как 3% самых популярных названий. Маленький размер связан с большим покрытием уже 3-мя процентами, а также с вычислительной сложностью задачи кластеризации 

In [4]:
unique_data, freq = np.unique(data, return_counts=True)
sorted_indexes = np.argsort(freq)[::-1]
unique_data = unique_data[sorted_indexes]

In [5]:
m = len(unique_data)
training_data = unique_data[:int(m * 0.03)]
n = len(training_data)
training_data

array(['Бухгалтер', 'Продавец', 'Водитель', ..., 'Зкономист', 'Продавац',
       'Изолировщик на теплоизоляции'], dtype=object)

Для начала подготовим данные для модели. Ключевых идей несколько:
- Будем анализировать биграммы, так как самые частые опечатки - неправильная буква или ошибка в порядке соседних букв (ввиду быстрого набора текста). Но такие изменения не очень сказываются на распределении биграмм. Соответственно кластеризировать мы будем вектора из частот биграмм

In [6]:
def preprocess(word):
    word = word.lower()
    save = 'йцкнгшщзхфвпрлджчсмтб'
    new_word = ''
    for ch in word:
        if ch in save:
            new_word += ch
    return new_word

def calculate_bigram(word):
    result = np.zeros(33 * 33, dtype='int')
    for j in range(len(word) - 1):
        x = ord(word[j]) - ord('а')
        y = ord(word[j + 1]) - ord('а')
        result[x * 33 + y] += 1
    return result

Сделаем препроцессинг каждого слова (выкинем все кроме согласных)

In [7]:
preprocessed_data = [''] * n
for i in tqdm(range(n)):
    preprocessed_data[i] = preprocess(training_data[i])

100%|█████████████████████████████████| 30060/30060 [00:00<00:00, 370261.71it/s]


Теперь посчитаем вектор частот биграмм для каждого слова

In [8]:
bigrams = np.array([np.zeros(33*33) for i in range(n)])
for i in tqdm(range(n)):
    bigrams[i] = calculate_bigram(preprocessed_data[i])

100%|█████████████████████████████████| 30060/30060 [00:00<00:00, 119374.67it/s]


Теперь создадим модель для кластеризации. Используем евклюдову метрику (по умолчанию), чтобы иметь возможность объединять кластеры (не вдаваясь в подробности работы самой модели), минимизируя дисперсию внутри кластера. Остальные гиперпараметры были подобраны во время подготовки.

In [9]:
model = AgglomerativeClustering(distance_threshold=5, linkage='ward', n_clusters=None).fit(bigrams)

Дальнейший алгоритм заключается в следующем:
- Для каждого кластера на основании **всех** данных выберем самый частовстречающийся - назовем его представителем
- Чтобы предсказать правильное написание найдем кластер с минимальным расстояниям **по всем** словам из кластера и возьмем представителя

In [10]:
def mse(a, b):
    return np.square(np.subtract(a, b)).mean()

class Cluster:
    def __init__(self):
        self.names = []
        self.bigrams = []
        self.best_ = 0

    def append(self, name, bigram):
        self.names.append(name)
        self.bigrams.append(bigram)

    def distance(self, bigram):
        if len(self.names) == 0:
            return float('inf')
        result = mse(bigram, self.bigrams[0])
        for i in range(1, len(self.bigrams)):
            result = min(result, mse(bigram, self.bigrams[i]))
        return result

    def fit(self, X):
        if len(self.names) == 0:
            return
        count = [0 for i in range(len(self.names))]
        for name in X:
            if name in self.names:
                count[self.names.index(name)] += 1
        self.best_ = count.index(max(count))

    def best(self):
        if len(self.names) == 0:
            return None
        return self.names[self.best_]


In [11]:
class MisspellModel:
    def __init__(self, n, labels, names, bigrams):
        self.clusters = [Cluster() for i in range(n)]
        for i in range(n):
            label = labels[i]
            self.clusters[label].append(names[i], bigrams[i])

    def fit(self, X):
        for i in tqdm(range(len(self.clusters))):
            self.clusters[i].fit(X)

    def predict(self, word):
        x = calculate_bigram(preprocess(word))
        dist = [0 for i in range(len(self.clusters))]
        for i in range(len(self.clusters)):
            dist[i] = self.clusters[i].distance(x)
        return self.clusters[dist.index(min(dist))].best()

In [12]:
predicter = MisspellModel(model.n_clusters_, model.labels_, training_data, bigrams)

In [13]:
predicter.fit(data)

100%|█████████████████████████████████████| 4801/4801 [2:20:08<00:00,  1.75s/it]


Пример работы предсказателя

In [16]:
predicter.predict('Сарщик')

'Сварщик'

In [18]:
predicter.predict('по работе с населением')

'Менеджер по работе с населением'

In [25]:
predicter.predict('родавец')

'Продавец'

In [29]:
predicter.predict('Водитеь')

'Водитель'

In [36]:
# df_TRAIN_RES_1 = pd.read_csv(f'{data_path}/TRAIN_RES_1.csv') # всего их 5
df = pd.read_csv('./TEST_SAL.csv')

In [45]:
df['company_code']

0       1087746619487
1       1137746734399
2       1082360000106
3       1024201368740
4       1033231004223
            ...      
1951    1021300928703
1952    1026901734265
1953    1022601320631
1954    1207800127875
1955    1022302836654
Name: company_code, Length: 1956, dtype: object